# Efficient Sudoku Solver

### Candidate Initialization & Board State Representation

This section defines the data structures and functions used to initialize candidate numbers for each cell in the Sudoku board:

- `box_index(r, c)`: Computes the box index (0–8) for a given cell.
- `init_candidates(board)`: Creates a 9×9 grid of sets representing possible candidates for each cell.

In [91]:
def box_index(r, c):
    return (r // 3) * 3 + (c // 3)

def init_candidates(board):
    digits = set(range(1, 10))
    rows = [set() for _ in range(9)]
    cols = [set() for _ in range(9)]
    boxes = [set() for _ in range(9)]

    for i in range(9):
        for j in range(9):
            v = board[i][j]
            if v != 0:
                rows[i].add(v)
                cols[j].add(v)
                boxes[box_index(i, j)].add(v)

    candidates = [[set() for _ in range(9)] for _ in range(9)]
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0: 
                used = rows[i] | cols[j] | boxes[box_index(i, j)]
                candidates[i][j] = digits - used
            else:  
                candidates[i][j] = {board[i][j]}
    return candidates

In [92]:
board1 = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [93]:
candidates = init_candidates(board1)
for row in candidates:
    print(row)

[{5}, {3}, {1, 2, 4}, {2, 6}, {7}, {8, 2, 4, 6}, {8, 1, 4, 9}, {1, 2, 4, 9}, {8, 2, 4}]
[{6}, {2, 4, 7}, {2, 4, 7}, {1}, {9}, {5}, {8, 3, 4, 7}, {2, 3, 4}, {8, 2, 4, 7}]
[{1, 2}, {9}, {8}, {2, 3}, {3, 4}, {2, 4}, {1, 3, 4, 5, 7}, {6}, {2, 4, 7}]
[{8}, {1, 2, 5}, {1, 2, 5, 9}, {9, 5, 7}, {6}, {1, 4, 7}, {9, 4, 5, 7}, {9, 2, 4, 5}, {3}]
[{4}, {2, 5}, {9, 2, 5, 6}, {8}, {5}, {3}, {9, 5, 7}, {9, 2, 5}, {1}]
[{7}, {1, 5}, {1, 3, 5, 9}, {9, 5}, {2}, {1, 4}, {8, 9, 4, 5}, {9, 4, 5}, {6}]
[{1, 3, 9}, {6}, {1, 3, 4, 5, 7, 9}, {3, 5, 7}, {3, 5}, {7}, {2}, {8}, {4}]
[{2, 3}, {8, 2, 7}, {2, 3, 7}, {4}, {1}, {9}, {3, 6}, {3}, {5}]
[{1, 2, 3}, {1, 2, 4, 5}, {1, 2, 3, 4, 5}, {2, 3, 5, 6}, {8}, {2, 6}, {1, 3, 4, 6}, {7}, {9}]


In [94]:
def generate_units():
    units = []

    # Rows
    for r in range(9):
        units.append([(r, c) for c in range(9)])

    # Columns
    for c in range(9):
        units.append([(r, c) for r in range(9)])

    # Boxes
    for box_row in range(3):
        for box_col in range(3):
            units.append([
                (r, c)
                for r in range(box_row * 3, box_row * 3 + 3)
                for c in range(box_col * 3, box_col * 3 + 3)
            ])
    return units

units = generate_units()
print(f"Total units: {len(units)}") 
print("First row unit:", units[0])
print("First box unit:", units[18])

# Total units = 27 (9 rows + 9 columns + 9 boxes).

# Indexing order:
# 0–8 → rows
# 9–17 → columns
# 18–26 → boxes

Total units: 27
First row unit: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8)]
First box unit: [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


### Candidate Elimination After Assignment

This function updates the candidate sets after a number is placed in a cell:

- `eliminate_from_others(candidates, row, col, val)`:
  - Removes `val` from all candidate sets in the same row, column, and 3×3 box, except the cell itegies.


In [95]:
def eliminate_from_others(candidates, row, col, val):
 
    for c in range(9):
        if c != col:
            candidates[row][c].discard(val)

    for r in range(9):
        if r != row:
            candidates[r][col].discard(val)

    box_r, box_c = row // 3 * 3, col // 3 * 3
    for r in range(box_r, box_r + 3):
        for c in range(box_c, box_c + 3):
            if (r, c) != (row, col):
                candidates[r][c].discard(val)

### Phase 1: Human Strategies Solver

This function implements the **first phase** of the Sudoku solver using human-style logical strategies

In [96]:
def naked_singles(board, candidates):
    changed = False  

    for i in range(9):
        for j in range(9):
            if board[i][j] == 0 and len(candidates[i][j]) == 1:
                val = next(iter(candidates[i][j]))
                board[i][j] = val
                candidates[i][j] = {val}  
                eliminate_from_others(candidates, i, j, val)
                changed = True

    return changed  

In [97]:
def hidden_singles(board, candidates, units):
    changed = False 

    for unit in units:
        pos_map = {n: [] for n in range(1, 10)}
        for (r, c) in unit:
            if board[r][c] == 0:
                for num in candidates[r][c]:
                    pos_map[num].append((r, c))

        for num, positions in pos_map.items():
            if len(positions) == 1:
                r, c = positions[0]
                board[r][c] = num
                candidates[r][c] = {num}  
                eliminate_from_others(candidates, r, c, num)
                changed = True

    return changed

In [98]:
def naked_n(board, candidates, units, max_n=4):
    changed = False
    for N in range(2, max_n + 1):
        for unit in units:
            candidate_map = {}
            for r, c in unit:
                if board[r][c] == 0 and 1 <= len(candidates[r][c]) <= N:
                    key = tuple(sorted(candidates[r][c]))
                    candidate_map.setdefault(key, []).append((r, c))
            
            for key, positions in candidate_map.items():
                if len(key) == len(positions) == N:  
                    for r, c in unit:
                        if (r, c) not in positions and board[r][c] == 0:
                            before = len(candidates[r][c])
                            candidates[r][c] -= set(key)
                            if len(candidates[r][c]) == 1:
                                val = next(iter(candidates[r][c]))
                                board[r][c] = val
                                eliminate_from_others(candidates, r, c, val)
                                changed = True
                            elif len(candidates[r][c]) < before:
                                changed = True
    return changed

- `solve_first_phase(board)`:
  - Initializes candidate sets for each cell using `init_candidates`.
  - Generates all units (rows, columns, boxes) using `generate_units`.
  - Iteratively applies deterministic strategies until no further progress:
    - **Naked Singles**: Fill cells with only one candidate.
    - **Hidden Singles**: Fill numbers that appear in only one cell within a unit.
    - **Naked N-tuples**: Remove candidates based on pairs, triples, or quads within a unit (`max_n=4` by default).
  - Returns the updated board and candidate sets for Phase 2.

In [99]:
def solve_first_phase(board):

    candidates = init_candidates(board)
    units = generate_units()
    
    changed = True
    while changed:
        changed = False
        
        # Naked Singles
        if naked_singles(board, candidates):
            changed = True
        
        # Hidden Singles
        if hidden_singles(board, candidates, units):
            changed = True
        
        # Naked 
        if naked_n(board, candidates, units, max_n=4):
            changed = True
    
    return board, candidates

In [100]:
board17 = [
    [0,4,0,0,0,0,0,8,0],
    [0,0,7,0,0,0,0,6,0],
    [0,0,0,0,1,0,0,0,0],
    [4,1,0,0,0,0,2,0,0],
    [0,0,0,0,0,5,0,0,0],
    [0,3,0,0,0,0,0,0,0],
    [0,0,6,0,0,7,0,0,3],
    [0,0,5,8,0,6,0,0,0],
    [0,0,0,0,0,0,0,0,1]
]

In [82]:
solve_first_phase(board17)

[[6, 4, 1, 5, 7, 2, 3, 8, 9],
 [2, 5, 7, 9, 8, 3, 1, 6, 4],
 [8, 9, 3, 6, 1, 4, 5, 2, 7],
 [4, 1, 9, 7, 6, 8, 2, 3, 5],
 [7, 6, 2, 3, 9, 5, 4, 1, 8],
 [5, 3, 8, 4, 2, 1, 7, 9, 6],
 [9, 2, 6, 1, 4, 7, 8, 5, 3],
 [1, 7, 5, 8, 3, 6, 9, 4, 2],
 [3, 8, 4, 2, 5, 9, 6, 7, 1]]

In [101]:
board2 = [
    [0, 0, 3, 0, 2, 0, 6, 0, 0],
    [9, 0, 0, 3, 0, 5, 0, 0, 1],
    [0, 0, 1, 8, 0, 6, 4, 0, 0],
    [0, 0, 8, 1, 0, 2, 9, 0, 0],
    [7, 0, 0, 0, 0, 0, 0, 0, 8],
    [0, 0, 6, 7, 0, 8, 2, 0, 0],
    [0, 0, 2, 6, 0, 9, 5, 0, 0],
    [8, 0, 0, 2, 0, 3, 0, 0, 9],
    [0, 0, 5, 0, 1, 0, 3, 0, 0]
]

In [84]:
solve_first_phase(board2)

[[4, 8, 3, 9, 2, 1, 6, 5, 7],
 [9, 6, 7, 3, 4, 5, 8, 2, 1],
 [2, 5, 1, 8, 7, 6, 4, 9, 3],
 [5, 4, 8, 1, 3, 2, 9, 7, 6],
 [7, 2, 9, 5, 6, 4, 1, 3, 8],
 [1, 3, 6, 7, 9, 8, 2, 4, 5],
 [3, 7, 2, 6, 8, 9, 5, 1, 4],
 [8, 1, 4, 2, 5, 3, 7, 6, 9],
 [6, 9, 5, 4, 1, 7, 3, 8, 2]]

In [102]:
board3 = [
    [0, 0, 0, 0, 0, 0, 2, 0, 0],
    [0, 8, 0, 0, 0, 7, 0, 9, 0],
    [6, 0, 2, 0, 0, 0, 5, 0, 0],
    [0, 7, 0, 0, 6, 0, 0, 0, 0],
    [0, 0, 0, 9, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 2, 0, 0, 4, 0],
    [0, 0, 5, 0, 0, 0, 6, 0, 3],
    [0, 9, 0, 4, 0, 0, 0, 7, 0],
    [0, 0, 6, 0, 0, 0, 0, 0, 0]
]

In [86]:
solve_first_phase(board3)

[[9, 0, 7, 6, 0, 0, 2, 0, 0],
 [0, 8, 0, 2, 0, 7, 0, 9, 6],
 [6, 0, 2, 0, 0, 0, 5, 0, 7],
 [0, 7, 0, 0, 6, 0, 0, 0, 0],
 [0, 0, 0, 9, 0, 1, 0, 6, 0],
 [0, 6, 0, 0, 2, 0, 0, 4, 0],
 [0, 0, 5, 0, 0, 0, 6, 0, 3],
 [0, 9, 0, 4, 0, 6, 0, 7, 0],
 [0, 0, 6, 0, 0, 0, 0, 0, 0]]

In [103]:
boardX = [
    [0,0,0,8,0,1,0,0,0],
    [0,0,0,0,0,0,4,3,0],
    [5,0,0,0,0,0,0,0,0],
    [0,0,0,0,7,0,8,0,0],
    [0,0,0,0,0,0,1,0,0],
    [0,2,0,0,3,0,0,0,0],
    [6,0,0,0,0,0,0,7,5],
    [0,0,3,4,0,0,0,0,0],
    [0,0,0,2,0,0,6,0,0]
]

In [88]:
solve_first_phase(boardX)

[[0, 0, 0, 8, 0, 1, 0, 0, 0],
 [0, 0, 0, 7, 0, 5, 4, 3, 0],
 [5, 0, 0, 3, 0, 6, 0, 0, 0],
 [0, 0, 0, 0, 7, 0, 8, 0, 0],
 [0, 0, 0, 0, 8, 0, 1, 0, 0],
 [0, 2, 0, 0, 3, 0, 0, 0, 0],
 [6, 4, 2, 9, 1, 8, 3, 7, 5],
 [0, 5, 3, 4, 6, 7, 0, 0, 0],
 [0, 0, 0, 2, 5, 3, 6, 0, 0]]

### Phase 2: MRV Backtracking Solver

This function implements the **second phase** of the Sudoku solver using **recursive backtracking with MRV (Minimum Remaining Values)**:

In [104]:
def select_mrv_cell(board, candidates):
    min_len = 10
    target_cell = None
    for r in range(9):
        for c in range(9):
            if board[r][c] == 0:
                cand_len = len(candidates[r][c])
                if cand_len < min_len:
                    min_len = cand_len
                    target_cell = (r, c)
    return target_cell

- `solve_second_phase(board)`:
  - Propagates **Phase 1** first to fill all deterministic cells.
  - Uses **MRV heuristic** to select the empty cell with the fewest candidates.
  - Tries each candidate for that cell:
    - Makes a **copy of the board and candidates**.
    - Assigns the candidate and updates the peers using `eliminate_from_others`.
    - Propagates **Phase 1** again to capture new deterministic opportunities.
    - Recursively calls itself to continue solving.
    - If a solution is found, returns the solved board.
  - If no candidate works, returns `False`.

In [109]:
def solve_second_phase(board):
    units = generate_units()
    
    board, candidates = solve_first_phase(board)

    cell = select_mrv_cell(board, candidates)
    if not cell:
        return board 

    r, c = cell
    for val in sorted(candidates[r][c]):
        board_copy = [row[:] for row in board]
        candidates_copy = [row[:] for row in candidates]

        board_copy[r][c] = val
        candidates_copy[r][c] = {val}
        eliminate_from_others(candidates_copy, r, c, val)

        board_copy, candidates_copy = solve_first_phase(board_copy)

        solved = solve_second_phase(board_copy)
        if solved:  
            return solved

    return False  

In [111]:
solve_second_phase(board1)

[[5, 3, 4, 6, 7, 8, 9, 1, 2],
 [6, 7, 2, 1, 9, 5, 3, 4, 8],
 [1, 9, 8, 3, 4, 2, 5, 6, 7],
 [8, 5, 9, 7, 6, 1, 4, 2, 3],
 [4, 2, 6, 8, 5, 3, 7, 9, 1],
 [7, 1, 3, 9, 2, 4, 8, 5, 6],
 [9, 6, 1, 5, 3, 7, 2, 8, 4],
 [2, 8, 7, 4, 1, 9, 6, 3, 5],
 [3, 4, 5, 2, 8, 6, 1, 7, 9]]

In [112]:
solve_second_phase(board17)

[[6, 4, 1, 5, 7, 2, 3, 8, 9],
 [2, 5, 7, 9, 8, 3, 1, 6, 4],
 [8, 9, 3, 6, 1, 4, 5, 2, 7],
 [4, 1, 9, 7, 6, 8, 2, 3, 5],
 [7, 6, 2, 3, 9, 5, 4, 1, 8],
 [5, 3, 8, 4, 2, 1, 7, 9, 6],
 [9, 2, 6, 1, 4, 7, 8, 5, 3],
 [1, 7, 5, 8, 3, 6, 9, 4, 2],
 [3, 8, 4, 2, 5, 9, 6, 7, 1]]

In [113]:
solve_second_phase(board2)

[[4, 8, 3, 9, 2, 1, 6, 5, 7],
 [9, 6, 7, 3, 4, 5, 8, 2, 1],
 [2, 5, 1, 8, 7, 6, 4, 9, 3],
 [5, 4, 8, 1, 3, 2, 9, 7, 6],
 [7, 2, 9, 5, 6, 4, 1, 3, 8],
 [1, 3, 6, 7, 9, 8, 2, 4, 5],
 [3, 7, 2, 6, 8, 9, 5, 1, 4],
 [8, 1, 4, 2, 5, 3, 7, 6, 9],
 [6, 9, 5, 4, 1, 7, 3, 8, 2]]

In [114]:
solve_second_phase(board3)

[[9, 5, 7, 6, 1, 3, 2, 8, 4],
 [4, 8, 3, 2, 5, 7, 1, 9, 6],
 [6, 1, 2, 8, 4, 9, 5, 3, 7],
 [1, 7, 8, 3, 6, 4, 9, 5, 2],
 [5, 2, 4, 9, 7, 1, 3, 6, 8],
 [3, 6, 9, 5, 2, 8, 7, 4, 1],
 [8, 4, 5, 7, 9, 2, 6, 1, 3],
 [2, 9, 1, 4, 3, 6, 8, 7, 5],
 [7, 3, 6, 1, 8, 5, 4, 2, 9]]

In [115]:
solve_second_phase(boardX)

[[2, 3, 7, 8, 4, 1, 5, 6, 9],
 [8, 9, 6, 7, 2, 5, 4, 3, 1],
 [5, 1, 4, 3, 9, 6, 2, 8, 7],
 [3, 6, 1, 5, 7, 9, 8, 4, 2],
 [4, 7, 5, 6, 8, 2, 1, 9, 3],
 [9, 2, 8, 1, 3, 4, 7, 5, 6],
 [6, 4, 2, 9, 1, 8, 3, 7, 5],
 [1, 5, 3, 4, 6, 7, 9, 2, 8],
 [7, 8, 9, 2, 5, 3, 6, 1, 4]]

In [116]:
board0 = [
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0]
]

In [117]:
solve_second_phase(board0)

[[1, 2, 3, 4, 5, 6, 7, 8, 9],
 [4, 5, 6, 7, 8, 9, 1, 2, 3],
 [7, 8, 9, 1, 2, 3, 4, 5, 6],
 [2, 3, 1, 6, 7, 4, 8, 9, 5],
 [8, 7, 5, 9, 1, 2, 3, 6, 4],
 [6, 9, 4, 5, 3, 8, 2, 1, 7],
 [3, 1, 7, 2, 6, 5, 9, 4, 8],
 [5, 4, 2, 8, 9, 7, 6, 3, 1],
 [9, 6, 8, 3, 4, 1, 5, 7, 2]]

This phase guarantees solution for any valid Sudoku puzzle, combining **guessing** with **logic propagation**.